In [ ]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter6 import Starter
from synthpop.synthesizer import synthesize_all, synthesize_all_in_parallel_full, enable_logging 
import os
import pandas as pd
#enable_logging()

In [ ]:
census_key = os.environ["CENSUS"]
state_name = "MI"
county_name = "Oakland County"
starter = Starter(census_key, state_name, county_name)

In [ ]:
# make lookup series for HH and Person marginals
ps_m = {}
ps_m['h'] = pd.Series({('hh_age_of_head', 'ge25-lt45'):'HHAGE2',
 ('hh_age_of_head', 'ge45-lt65'):'HHAGE3',
 ('hh_age_of_head', 'ge65'):'HHAGE4',
 ('hh_age_of_head', 'lt25'):'HHAGE1',
 ('hh_cars', 'none'):'HHCAR0',
 ('hh_cars', 'one'):'HHCAR1',
 ('hh_cars', 'two or more'):'HHCAR2',
 ('hh_children', 'no'):'HHCHD2',
 ('hh_children', 'yes'):'HHCHD1',
 ('hh_income', 'gt100'):'HHINC4',
 ('hh_income', 'gt30-lt60'):'HHINC2',
 ('hh_income', 'gt60-lt100'):'HHINC3',
 ('hh_income', 'lt30'):'HHINC1',
 ('hh_race_of_head', 'asian'):'HHRACE3',
 ('hh_race_of_head', 'black'):'HHRACE2',
 ('hh_race_of_head', 'other'):'HHRACE4',
 ('hh_race_of_head', 'white'):'HHRACE1',
 ('hh_size', 'five'):'HHPERSONS5',
 ('hh_size', 'four'):'HHPERSONS4',
 ('hh_size', 'one'):'HHPERSONS1',
 ('hh_size', 'seven or more'):'HHPERSONS7',
 ('hh_size', 'six'):'HHPERSONS6',
 ('hh_size', 'three'):'HHPERSONS3',
 ('hh_size', 'two'):'HHPERSONS2',
 ('hh_workers', 'none'):'HHWORKER0',
 ('hh_workers', 'one'):'HHWORKER1',
 ('hh_workers', 'two or more'):'HHWORKER2',
 ('hispanic_head', 'no'):'HHHISP1',
 ('hispanic_head', 'yes'):'HHHISP2'})

ps_m['p'] = pd.Series({('person_age', '17 and under'):'AGEP1',
 ('person_age', '18 to 64'):'AGEP2',
 ('person_age', '65 and up'):'AGEP3',
 ('person_sex', 'female'):'SEX2',
 ('person_sex', 'male'):'SEX1',
 ('race', 'asian'):'RACE3',
 ('race', 'black'):'RACE2',
 ('race', 'other'):'RACE4',
 ('race', 'white'):'RACE1'})

In [ ]:
indexes = starter.get_available_geography_ids()
#geog_ids = []
dict_marg = {'h':[], 'p':[]}

for i, geog_id in enumerate(indexes):
    #geog_ids.append(geog_id)
    dict_marg['h'].append(starter.get_household_marginal_for_geography(geog_id))
    dict_marg['p'].append(starter.get_person_marginal_for_geography(geog_id))


In [ ]:
dict_total = {'h':"HHBASE", 'p':"POPBASE"}
dict_m = {}

for m in ['h', 'p']:
    df_m = pd.concat(dict_marg[m], axis=1)
    ps_t = df_m.sum(level=0).mean().round(0)
    df_m['controls'] = ps_m[m].values
    df_m = df_m.reset_index(drop=True).set_index('controls').T
    df_m.index.set_names(['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE'], inplace = True)
    df_m[dict_total[m]]=ps_t.values
    dict_m[m] = df_m



In [ ]:
df_blkgrp = pd.concat(dict_m.values(), axis = 1)
df_blkgrp['BLKGRP'] = [ ''.join(x) for x in df_blkgrp.index.to_numpy()] # get unique BLKGRP 
df_blkgrp.to_csv('control_totals_' + county_name + '_blkgrp.csv')




In [ ]:
df_blkgrp['TRACT'] = df_blkgrp['BLKGRP'].str[:-1]
df_blkgrp.drop('BLKGRP',axis=1,inplace=True)
df_tract = df_blkgrp.groupby("TRACT").sum()
df_tract.to_csv('control_totals_' + county_name + '_tract.csv')


In [ ]:
df_blkgrp['COUNTY'] = df_blkgrp.index.get_level_values(1)

In [ ]:
df_region = df_blkgrp.groupby('COUNTY').sum()
df_region.to_csv('control_totals_' + county_name + '_region.csv')